In [5]:
# importing required packages

#%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import IPython
import csv
import pandas as pd

import matplotlib #scientific plots
import matplotlib.pyplot as plt

import numpy as np # numerical programming

import sklearn #scikit-learn
import math #Mathematical functions
import pylab 
import time
import pprint as pp
from mpl_toolkits.mplot3d import Axes3D #we use this to plot 3d
from matplotlib.ticker import NullFormatter #No labels on the ticks.
from IPython.display import Image
#from __future__ import division
first_array=np.genfromtxt('data_want.csv',delimiter=',')
SFE_data = first_array
SFE_data = np.array(SFE_data, dtype='float')
#SFE-(1-error),each rows
lower = SFE_data[:,10]*(1 - SFE_data[:,11]) 
higher = SFE_data[:,10]*(1 + SFE_data[:,11])
lowhigh = np.column_stack((lower,higher))
SFE_data = np.column_stack((SFE_data,lowhigh))
#print(SFE_data.shape)
#print(SFE_data[0,:])
SFE_classes = np.zeros(SFE_data.shape[0])

SFE_classes[SFE_data[:,10] <= 20] = 1
SFE_classes[(SFE_data[:,10] > 20) & (SFE_data[:,10] <= 45)] = 2
SFE_classes[(SFE_data[:,10] > 45)] = 3

#check number of entries for each class 
#print((SFE_classes == 1).sum())
#print((SFE_classes == 2).sum())
#print((SFE_classes == 3).sum())

#check if only the needed classes there
print((SFE_classes == 1).sum() + (SFE_classes == 2).sum() + (SFE_classes == 3).sum()) 
#print(SFE_classes.shape)
SFE_data.shape
data_SFE=np.genfromtxt('data_nor.csv',delimiter=',')
# save to csv file
np.savetxt('data_nor.csv', SFE_data[:,:10], delimiter=',')

#normalization and standardization
from sklearn import preprocessing
#normalization = preprocessing.normalize(SFE_data[:,:10], axis=0)
scaler = preprocessing.MinMaxScaler()
normalization = scaler.fit_transform(SFE_data[:,:10])
#print(normalization[0])
#print(normalization.shape)


standardization = preprocessing.scale(SFE_data[:,:10], axis=0)

trying=SFE_data[:,:10]
#defining master indices
master_indices = np.arange(normalization.shape[0])
#print 'master indices shape : ', master_indices.shape

#choosing test set and corressponding training set
np.random.seed(25)
test_prop = 0.2
test_indices = np.random.choice(normalization.shape[0],int(math.floor(test_prop*normalization.shape[0])),replace = False)
test_x = normalization[test_indices]
test_y = SFE_classes[test_indices]

train_indices = master_indices
#print 'train indices shape :', train_indices.shape
for item in test_indices:
    train_indices = np.delete(train_indices,np.where(train_indices==item))
#print 'train indices shape  :', train_indices.shape

train_x = normalization[train_indices]
train_y = SFE_classes[train_indices]
#print('Train', train_x.shape, train_y.shape)
#print('Test', test_x.shape, test_y.shape)
#print(test_y[73])

374


In [6]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers
from sklearn.metrics import confusion_matrix

def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=5.0):
    return np.exp(-linalg.norm(x-y)**2 / (2 * (sigma ** 2)))
class SVM(object):

    def __init__(self, kernel=polynomial_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # Gram matrix
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = self.kernel(X[i], X[j])

        P = cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        b = cvxopt.matrix(0.0)

        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))

        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)

        # Lagrange multipliers
        a = np.ravel(solution['x'])

        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-15
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X[sv]
        self.sv_y = y[sv]
        print ("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * K[ind[n],sv])
        self.b /= len(self.a)

        # Weight vector
        if self.kernel == linear_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        else:
            self.w = None

    def project(self, X):
        if self.w is not None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * self.kernel(X[i], sv)
                y_predict[i] = s
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

if __name__ == "__main__":
    import pylab as pl

    # def gen_lin_separable_data():
    #     # generate training data in the 2-d case
    #     mean1 = np.array([0, 2])
    #     mean2 = np.array([2, 0])
    #     cov = np.array([[0.8, 0.6], [0.6, 0.8]])
    #     X1 = np.random.multivariate_normal(mean1, cov, 100)
    #     y1 = np.ones(len(X1))
    #     X2 = np.random.multivariate_normal(mean2, cov, 100)
    #     y2 = np.ones(len(X2)) * -1
    #     return X1, y1, X2, y2

    # def gen_non_lin_separable_data():
    #     mean1 = [-1, 2]
    #     mean2 = [1, -1]
    #     mean3 = [4, -4]
    #     mean4 = [-4, 4]
    #     cov = [[1.0,0.8], [0.8, 1.0]]
    #     X1 = np.random.multivariate_normal(mean1, cov, 50)
    #     X1 = np.vstack((X1, np.random.multivariate_normal(mean3, cov, 50)))
    #     y1 = np.ones(len(X1))
    #     X2 = np.random.multivariate_normal(mean2, cov, 50)
    #     X2 = np.vstack((X2, np.random.multivariate_normal(mean4, cov, 50)))
    #     y2 = np.ones(len(X2)) * -1
    #     return X1, y1, X2, y2

    # def gen_lin_separable_overlap_data():
    #     # generate training data in the 2-d case
    #     mean1 = np.array([0, 2])
    #     mean2 = np.array([2, 0])
    #     cov = np.array([[1.5, 1.0], [1.0, 1.5]])
    #     X1 = np.random.multivariate_normal(mean1, cov, 100)
    #     y1 = np.ones(len(X1))
    #     X2 = np.random.multivariate_normal(mean2, cov, 100)
    #     y2 = np.ones(len(X2)) * -1
    #     return X1, y1, X2, y2

    # def split_train(X1, y1, X2, y2):
    #     X1_train = X1[:90]
    #     y1_train = y1[:90]
    #     X2_train = X2[:90]
    #     y2_train = y2[:90]
    #     X_train = np.vstack((X1_train, X2_train))
    #     y_train = np.hstack((y1_train, y2_train))
    #     return X_train, y_train

    # def split_test(X1, y1, X2, y2):
    #     X1_test = X1[90:]
    #     y1_test = y1[90:]
    #     X2_test = X2[90:]
    #     y2_test = y2[90:]
    #     X_test = np.vstack((X1_test, X2_test))
    #     y_test = np.hstack((y1_test, y2_test))
    #     return X_test, y_test

    # def plot_margin(X1_train, X2_train, clf):
    #     def f(x, w, b, c=0):
    #         # given x, return y such that [x,y] in on the line
    #         # w.x + b = c
    #         return (-w[0] * x - b + c) / w[1]

    #     pl.plot(X1_train[:,0], X1_train[:,1], "ro")
    #     pl.plot(X2_train[:,0], X2_train[:,1], "bo")
    #     pl.scatter(clf.sv[:,0], clf.sv[:,1], s=100, c="g")

    #     # w.x + b = 0
    #     a0 = -4; a1 = f(a0, clf.w, clf.b)
    #     b0 = 4; b1 = f(b0, clf.w, clf.b)
    #     pl.plot([a0,b0], [a1,b1], "k")

    #     # w.x + b = 1
    #     a0 = -4; a1 = f(a0, clf.w, clf.b, 1)
    #     b0 = 4; b1 = f(b0, clf.w, clf.b, 1)
    #     pl.plot([a0,b0], [a1,b1], "k--")

    #     # w.x + b = -1
    #     a0 = -4; a1 = f(a0, clf.w, clf.b, -1)
    #     b0 = 4; b1 = f(b0, clf.w, clf.b, -1)
    #     pl.plot([a0,b0], [a1,b1], "k--")

    #     pl.axis("tight")
    #     pl.show()

    # def plot_contour(X1_train, X2_train, clf):
    #     pl.plot(X1_train[:,0], X1_train[:,1], "ro")
    #     pl.plot(X2_train[:,0], X2_train[:,1], "bo")
    #     pl.scatter(clf.sv[:,0], clf.sv[:,1], s=100, c="g")

    #     X1, X2 = np.meshgrid(np.linspace(-6,6,50), np.linspace(-6,6,50))
    #     X = np.array([[x1, x2] for x1, x2 in zip(np.ravel(X1), np.ravel(X2))])
    #     Z = clf.project(X).reshape(X1.shape)
    #     pl.contour(X1, X2, Z, [0.0], colors='k', linewidths=1, origin='lower')
    #     pl.contour(X1, X2, Z + 1, [0.0], colors='grey', linewidths=1, origin='lower')
    #     pl.contour(X1, X2, Z - 1, [0.0], colors='grey', linewidths=1, origin='lower')

    #     pl.axis("tight")
    #     pl.show()

    # def test_linear():
    #     X1, y1, X2, y2 = gen_lin_separable_data()
    #     X_train, y_train = split_train(X1, y1, X2, y2)
    #     X_test, y_test = split_test(X1, y1, X2, y2)

    #     clf = SVM()
    #     clf.fit(X_train, y_train)

    #     y_predict = clf.predict(X_test)
    #     correct = np.sum(y_predict == y_test)
    #     print ("%d out of %d predictions correct" % (correct, len(y_predict)))

    #     plot_margin(X_train[y_train==1], X_train[y_train==-1], clf)

    def test_non_linear():
        # X1, y1, X2, y2 = gen_non_lin_separable_data()
        # X_train, y_train = split_train(X1, y1, X2, y2)
        # X_test, y_test = split_test(X1, y1, X2, y2)

        clf = SVM(polynomial_kernel)
        clf.fit(train_x, train_y)

        y_predict = clf.predict(test_x)
        correct = np.sum(y_predict == test_y)
        cm_svc = confusion_matrix(test_y, y_predict)
        print("Confusion Matrix \n",confusion_matrix(test_y,y_predict),"\n")
        #print ("%d out of %d predictions correct" % (correct, len(y_predict)))

        #plot_contour(X_train[y_train==1], X_train[y_train==-1], clf)

    def test_soft():
        # X1, y1, X2, y2 = gen_lin_separable_overlap_data()
        # X_train, y_train = split_train(X1, y1, X2, y2)
        # X_test, y_test = split_test(X1, y1, X2, y2)

        clf = SVM(C=1)
        clf.fit(train_x, train_y)

        y_predict = clf.predict(test_x)
        correct = np.sum(y_predict == test_y)
        print ("%d out of %d predictions correct" % (correct, len(y_predict)))
        cm_svc = confusion_matrix(test_y, y_predict)
        print("Confusion Matrix \n",confusion_matrix(test_y,y_predict),"\n")

        #plot_contour(X_train[y_train==1], X_train[y_train==-1], clf)
    test_soft()
    #from sklearn.metrics import confusion_matrix
    #cm_svc = confusion_matrix(y_test,y_predict)
    #print "Confusion Matrix \n",confusion_matrix(y_test,y_predict),"\n"
 

     pcost       dcost       gap    pres   dres
 0: -1.1383e+01 -4.7529e+02  2e+03  2e+00  4e-14
 1: -6.3290e+00 -2.2231e+02  4e+02  3e-01  3e-14
 2: -1.5997e+00 -5.3496e+01  9e+01  5e-02  2e-14
 3: -2.9144e-01 -7.9450e+00  1e+01  8e-03  8e-15
 4: -5.6540e-02 -1.2330e+00  2e+00  1e-03  5e-15
 5: -1.0284e-02 -1.9926e-01  3e-01  2e-04  3e-15
 6: -3.1136e-04 -5.7053e-03  8e-03  4e-06  4e-15
 7: -3.1452e-06 -5.7126e-05  8e-05  4e-08  5e-15
 8: -3.1455e-08 -5.7125e-07  8e-07  4e-10  5e-15
 9: -3.1455e-10 -5.7125e-09  8e-09  4e-12  9e-15
Optimal solution found.
89 support vectors out of 300 points
18 out of 74 predictions correct
Confusion Matrix 
 [[18  0  0]
 [42  0  0]
 [14  0  0]] 

